In [789]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

In [790]:
#change file name as needed
#file must have row headers (1 for experimental and 0 for control samples) in column 1 and column headers for each attribute in row 1
dftrain = pd.read_csv('data.txt', delimiter = "\t", low_memory=False)

sampleattributenum = 10
samplelist = list(dftrain.head())
samplelist = samplelist[1:(1 + sampleattributenum)]

dftest = dftrain.sample(frac = 0.3, replace = False).reset_index(drop = True)
dftrain = dftrain.drop(dftest.index)
dftrain = dftrain.sort_values(by = dftrain.columns[0])
dftrain = dftrain.transpose()
dftest = dftest.transpose()

In [791]:
ytrain = dftrain.iloc[0]
ytrain = list(ytrain)

ytest = dftest.iloc[0]
ytest = list(ytest)

In [792]:
#build tree around difference between attribute worst and means with respect to standard error (se)
dfdifference = pd.DataFrame()

for col in dftrain:
    temp = dftrain[col]
    tempdict = {}
    for i in range(sampleattributenum):
        condition = temp[0]
        attribute = samplelist[i]
        mean = temp[i + 1]
        se = temp[i + sampleattributenum]
        worst = temp[i + 2 * sampleattributenum]
        decisionvalue = worst - mean + se
        tempdict[attribute] = decisionvalue
    dfdifference = dfdifference.append(tempdict, ignore_index = True)

dfdifference.insert(0, "condition", ytrain)
dfdifference = dfdifference.transpose()
dfdifference.columns = dfdifference.iloc[0]
dfdifference.drop(index = dfdifference.index[0], axis = 0, inplace = True)

In [793]:
templist = []

for i, iter in dfdifference.iterrows():
    controllist = iter[0]
    experimentallist = iter[1]
    controlmean = controllist.mean()
    controlSE = controllist.sem()
    experimentalmean = experimentallist.mean()
    experimentalSE = experimentallist.sem()    
    if controlmean > experimentalmean:
        controldecisionpt = controlmean - controlSE
        experimentaldecisionpt = experimentalmean + experimentalSE
    if controlmean < experimentalmean:
        controldecisionpt = controlmean + controlSE
        experimentaldecisionpt = experimentalmean - experimentalSE  
    templist.append([i, experimentaldecisionpt, controldecisionpt])
    
dfdecision = pd.DataFrame(templist, columns = ['attribute', 'expdecpt', 'controldecpt'])

In [794]:
#apply decision points to training data
attdeclist = []

for col in dftrain:
    temp = dftrain[col]
    attdectotal = 0
    
    for i in range(sampleattributenum):
        condition = temp[0]
        attribute = samplelist[i]
        mean = temp[i + 1]
        se = temp[i + sampleattributenum]
        worst = temp[i + 2 * sampleattributenum]
        decisionvalue = worst - mean + se
        tempdecpt = dfdecision.iloc[i]
        
        if tempdecpt[1] > tempdecpt[2]:
            if decisionvalue > tempdecpt[1]:
                attdec = 1
            elif decisionvalue < tempdecpt[2]:
                attdec = 0
            else:
                attdec = 0.5
        if tempdecpt[1] < tempdecpt[2]:
            if decisionvalue < tempdecpt[1]:
                attdec = 1
            elif decisionvalue > tempdecpt[2]:
                attdec = 0
            else:
                attdec = 0.5
        attdectotal = attdectotal + attdec
    if attdectotal > 5:
        guess = 1
    else:
        guess = 0
    attdeclist.append(guess)

In [795]:
#compare training set results with actual values
correct = 0

for index in range(len(ytrain)):
    trainguess = attdeclist[index]
    trainreal = ytrain[index]
    if trainguess == trainreal:
        correct = correct + 1

accuracy = (correct / (len(ytrain)))*100
print('The decision tree is', accuracy, '% accurate on training data.')   

The decision tree is 88.69346733668341 % accurate on training data.


In [796]:
#apply decision points to test data
attdeclist = []

for col in dftest:
    temp = dftest[col]
    attdectotal = 0
    
    for i in range(sampleattributenum):
        condition = temp[0]
        attribute = samplelist[i]
        mean = temp[i + 1]
        se = temp[i + sampleattributenum]
        worst = temp[i + 2 * sampleattributenum]
        decisionvalue = worst - mean + se
        tempdecpt = dfdecision.iloc[i]
        
        if tempdecpt[1] > tempdecpt[2]:
            if decisionvalue > tempdecpt[1]:
                attdec = 1
            elif decisionvalue < tempdecpt[2]:
                attdec = 0
            else:
                attdec = 0.5
        if tempdecpt[1] < tempdecpt[2]:
            if decisionvalue < tempdecpt[1]:
                attdec = 1
            elif decisionvalue > tempdecpt[2]:
                attdec = 0
            else:
                attdec = 0.5
        attdectotal = attdectotal + attdec
    if attdectotal > 5:
        guess = 1
    else:
        guess = 0
    attdeclist.append(guess)

In [797]:
#compare test set results with actual values
correct = 0

for index in range(len(ytest)):
    testguess = attdeclist[index]
    testreal = ytest[index]
    if trainguess == trainreal:
        correct = correct + 1

accuracy = (correct / (len(ytest)))*100
print('The decision tree is', accuracy, '% accurate on test data.')   

The decision tree is 100.0 % accurate on test data.
